# Parámetros

In [1]:
year = 2022
week = 8
#path = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\Info_POS_clientes\Sell Out - No B2b\Cofarmen'
path = r'C:\Users\amartinezgo\genommalabinternacional\Francisco Jose Delfino - Info_POS_clientes\Sell Out - No B2b\Cofarmen'

# Librerias

In [2]:
import pandas as pd
from datetime import datetime
import pyodbc
import shutil
import os
from openpyxl import load_workbook

# Conexión

In [3]:
#Conexion a bases de datos
conn1 = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_DWH;'
                     'Trusted_Connection=yes;')

query_dates = '''
                SELECT TmpFecha
                      ,TmpSemanaAnioGenomma
                      ,TmpAnioSemanaGenomma

                  FROM dbo.Dim_Tiempo where TmpDiaSemana = 7 order by TmpID'''

In [4]:
#Nombres de los archivos a procesar
filename_mdza = '{0}_{1}_Cofarmen_MDZA.xls'
filename_sanj = '{0}_{1}_Cofarmen_SANJ.xls'

file_names = [filename_mdza.format(year, str(week).zfill(2))] + [filename_sanj.format(year, str(week).zfill(2))]

In [5]:
file_names

['2022_08_Cofarmen_MDZA.xls', '2022_08_Cofarmen_SANJ.xls']

In [6]:
#VENTAS
dates = pd.read_sql(query_dates,conn1)
df_all_sales = pd.DataFrame()
files_to_check = []

for file in file_names:
    try:
        #Obtener el año y semana del nombre del archivo
        year = int(file.split('_')[0])
        week = int(file.split('_')[1])
        
        #Revisar con que sucursal se esta trabajando
        sucursal = file.split('_')[3].split('.')[0]
        if sucursal == 'MDZA':
            nombre_suc = '88158 Cofarmen - Belgrano 1294, Godoy Cruz'
            cod_suc = '88158'
    
        else:
            nombre_suc = 'Cofarmen San Juan - Genérico'
            cod_suc = 'SANJ_01'
        
        #Leer datos
        df = pd.read_csv(path + '\\' + file, encoding='latin1', error_bad_lines=False, sep='\t')
        df.dropna(how='all',inplace=True)

        df.columns = [i.strip() for i in df.columns]

        #df.fillna(value={'EAN':0},inplace=True)
        
        data_final = pd.DataFrame()
        
        #Crear dataframe final con las columnas correspondientes
        data_final['Unidades'] = df['VENTA']
        data_final['Fecha_Venta'] = dates[(dates['TmpAnioSemanaGenomma'] == year) & (dates['TmpSemanaAnioGenomma'] == week)]['TmpFecha'].unique()[0]
        data_final['Fecha_Venta'] = data_final['Fecha_Venta'].dt.date
        data_final['Cadena'] = 'Cofarmed'
        data_final['Cod_Prod de la Cadena'] = df['MATERIAL CODIGO'].astype('int64')
        data_final['EAN o UPC Genomma'] = df['EAN'].astype('int64')
        data_final['Descripcion_Prod'] = df['MATERIAL DESCRIPCION']
        data_final['Cod_Local'] = cod_suc
        data_final['Descripcion_Local'] = nombre_suc
        
        #Reordenar el dataframe y hacer ultimas modificaciones
        data_final = data_final[['Fecha_Venta','Cadena','Cod_Prod de la Cadena','EAN o UPC Genomma','Descripcion_Prod','Cod_Local','Descripcion_Local','Unidades']]
        data_final.dropna(subset=['Unidades'],inplace=True)
        data_final['Descripcion_Prod'] = data_final['Descripcion_Prod'].apply(lambda x: x.strip())
        df_all_sales = pd.concat([df_all_sales,data_final])
        print('Listo archivo ' + file)
        
    except:
        #En caso de haber archivos con un layout diferente o que se genere un error al correr el codigo se guarda el nombre 
        #en una lista para su revision
        files_to_check.append(file)


Listo archivo 2022_08_Cofarmen_MDZA.xls
Listo archivo 2022_08_Cofarmen_SANJ.xls


In [7]:
#Si no aparece vacio quiere decir que hubo un error con algun archivo y se mostrara el nombre
files_to_check

[]

In [8]:
#STOCK
dates = pd.read_sql(query_dates,conn1)
df_all_stock = pd.DataFrame()
files_to_check = []

for file in file_names:
    try:
        #Obtener el año y semana del nombre del archivo
        year = int(file.split('_')[0])
        week = int(file.split('_')[1])
        
        #Revisar con que sucursal se esta trabajando
        sucursal = file.split('_')[3].split('.')[0]
        if sucursal == 'MDZA':
            nombre_suc = '88158 Cofarmen - Belgrano 1294, Godoy Cruz'
            cod_suc = '88158'
    
        else:
            nombre_suc = 'Cofarmen San Juan - Genérico'
            cod_suc = 'SANJ_01'
        
        #Leer datos
        df = pd.read_csv(path + '//'+file, encoding='latin1', error_bad_lines=False, sep='\t')
        df.dropna(how='all',inplace=True)

        df.columns = [i.strip() for i in df.columns]

        #df.fillna(value={'EAN':0},inplace=True)

        data_final = pd.DataFrame()
        
        #Crear dataframe final con las columnas correspondientes
        data_final['Unidades'] = df['STOCK']
        data_final['Fecha_Venta'] = dates[(dates['TmpAnioSemanaGenomma'] == year) & (dates['TmpSemanaAnioGenomma'] == week)]['TmpFecha'].unique()[0]
        data_final['Fecha_Venta'] = data_final['Fecha_Venta'].dt.date
        data_final['Cadena'] = 'Cofarmed'
        data_final['Cod_Prod de la Cadena'] = df['MATERIAL CODIGO'].astype('int64')
        data_final['EAN o UPC Genomma'] = df['EAN'].astype('int64')
        data_final['Descripcion_Prod'] = df['MATERIAL DESCRIPCION']
        data_final['Cod_Local'] = cod_suc
        data_final['Descripcion_Local'] = nombre_suc
        
        #Reordenar el dataframe y hacer ultimas modificaciones
        data_final = data_final[['Fecha_Venta','Cadena','Cod_Prod de la Cadena','EAN o UPC Genomma','Descripcion_Prod','Cod_Local','Descripcion_Local','Unidades']]
        data_final.dropna(subset=['Unidades'],inplace=True)
        data_final['Descripcion_Prod'] = data_final['Descripcion_Prod'].apply(lambda x: x.strip())
        df_all_stock = pd.concat([df_all_stock,data_final])
        print('Listo archivo ' + file)
        
    except:
        files_to_check.append(file)

Listo archivo 2022_08_Cofarmen_MDZA.xls
Listo archivo 2022_08_Cofarmen_SANJ.xls


In [9]:
#Si no aparece vacio quiere decir que hubo un error con algun archivo y se mostrara el nombre
files_to_check

[]

# Export

In [10]:
def write_layout(filename, df, sellout=True):
    writer = pd.ExcelWriter(filename, engine='openpyxl', mode='a')
    writer.book = load_workbook(filename)
    writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    if sellout == True:
        df.to_excel(writer, sheet_name='Ventas', startcol=1, startrow=18, index=False, header=None)
    else:
        df.to_excel(writer, sheet_name='Stock', startcol=1, startrow=18, index=False, header=None)
    writer.save()

In [11]:
original = '../../../../1Data/2Catalogue/toISV/Layout/Formato No B2B Orig.xlsx'
new_folder = '../../../../1Data/2Catalogue/toISV/{0}/S{1}/'.format(year, str(week).zfill(2))

if not os.path.exists(new_folder):
    os.mkdir(new_folder)

In [12]:
%%time
filename = 'Formato No B2B Cofarmen Sem ' + str(week).zfill(2)
path_export = new_folder + '/' + filename +'.xlsx'
shutil.copy(original, path_export)
write_layout(path_export, df_all_sales, True)
write_layout(path_export, df_all_stock, False)

Wall time: 5.44 s
